Import the data table to pd Data Frame

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# connect to sqlite3 database

# Replace with your database file path in Google Drive
db_path = 'my_database.db'

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

df_encoded = pd.read_sql("SELECT * FROM Loan_Approval", conn)
# Example query (replace with your actual query)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

conn.close()

[('raw_data',), ('Loan_Approval',), ('Interest_Rate_Features',), ('Property_Values',), ('Risk_classifier',)]


In [11]:
df_encoded.head()

,action_taken,purchaser_type,preapproval,reverse_mortgage,open-end_line_of_credit,loan_amount,loan_to_value_ratio,interest_rate,loan_term,negative_amortization,...,occupancy_type_2,occupancy_type_3,submission_of_application_2,initially_payable_to_institution_2,aus-1_2,aus-1_3,aus-1_4,aus-1_5,aus-1_6,aus-1_7
0,5,2,1,0,0,205000.0,80.0,7.000,360.0,0,...,0,0,0,0,0,0,0,0,1,0
1,5,2,1,0,0,235000.0,80.0,6.500,360.0,0,...,0,0,0,0,0,0,0,0,1,0
2,5,2,1,0,0,505000.0,80.0,5.625,360.0,0,...,0,0,0,0,0,0,0,0,1,0
3,5,2,1,0,0,685000.0,80.0,6.375,360.0,0,...,0,0,0,0,0,0,0,0,1,0
4,5,2,1,0,0,455000.0,80.0,5.500,360.0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
df_encoded.columns

Index(['action_taken', 'purchaser_type', 'preapproval', 'reverse_mortgage',
       'open-end_line_of_credit', 'loan_amount', 'loan_to_value_ratio',
       'interest_rate', 'loan_term', 'negative_amortization',
       'interest_only_payment', 'balloon_payment',
       'other_nonamortizing_features', 'property_value', 'total_units',
       'income', 'applicant_credit_score_type',
       'co-applicant_credit_score_type', 'applicant_age_above_62',
       'co-applicant_age_above_62', 'tract_population',
       'tract_minority_population_percent',
       'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage',
       'tract_owner_occupied_units', 'tract_median_age_of_housing_units',
       'derived_loan_product_type_FHA:First Lien',
       'derived_loan_product_type_FSA/RHS:First Lien',
       'derived_loan_product_type_VA:First Lien',
       'derived_dwelling_category_Multifamily:Site-Built',
       'derived_dwelling_category_Single Family (1-4 Units):Manufactured',
       'lo

Models

In [13]:
# Define target variable (Loan Approval)
X = df_encoded.drop(columns=['action_taken'])
y = df_encoded['action_taken']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

In [ ]:
# # Get feature importance
# importances = clf.feature_importances_
# indices = np.argsort(importances)[::-1]

# # Plot top 10 features
# plt.figure(figsize=(10,6))
# plt.title("Top 10 Feature Importances")
# plt.bar(range(10), importances[indices[:10]], align="center")
# plt.xticks(range(10), [X.columns[i] for i in indices[:10]], rotation=90)
# plt.show()

NameError: name 'clf' is not defined

XG Boost Classifier

In [16]:
# Initialize and train XGBoost model
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)

# Predictions
y_pred_xgb = xgb.predict(X_test)

# Evaluate Performance
print("🔹 XGBoost Results:")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

c:\Users\MOHIT\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:31:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 XGBoost Results:
Accuracy: 0.9297673067036234
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     67458
           1       0.71      0.18      0.29      2763
           2       0.97      0.97      0.97     23020
           3       0.79      0.93      0.85     17646
           4       0.66      0.31      0.42      6285
           5       1.00      1.00      1.00     16712
           6       0.71      0.84      0.77        76
           7       0.84      0.75      0.80       251

    accuracy                           0.93    134211
   macro avg       0.83      0.75      0.76    134211
weighted avg       0.92      0.93      0.92    134211



In [24]:
from collections import Counter

prediction_counts = Counter(y_pred_xgb.tolist())  # Convert to list first
print("Frequency distribution of predictions:")
print(prediction_counts)


Frequency distribution of predictions:
Counter({0: 69442, 2: 23187, 3: 20915, 5: 16712, 4: 2946, 1: 695, 7: 224, 6: 90})


In [10]:
import joblib
joblib.dump(xgb, 'loan_approval_model.pkl')

['loan_approval_model.pkl']